In [34]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from dateutil.parser import parse
from itertools import compress
from collections import OrderedDict
from datetime import date
import datetime
import re
from datetime import timedelta
import openpyxl
from sklearn.metrics import mean_squared_error
import plotly.express as px

import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Create figure with secondary y-axis

In [7]:
def convert_entire_DF_to_numeric(DF):
    for col in DF.columns:
        DF[col] = pd.to_numeric(DF[col], errors='ignore')
    return DF

def getTheLocalMinMax(df_col1):
    '''
    Code that computes takes the index of the dataframe where the values are either local minima or local maxima
    Parameter: df_col1 is a series whose index is datetime
    Returns a dictionary where the loc_min and loc_max are stored.
    '''
    
    df_col1 = pd.to_numeric(df_col1, errors='coerce')
    loc_min = (np.diff(np.sign(np.diff(df_col1))) > 0).nonzero()[0] + 1         # local min
    loc_max = (np.diff(np.sign(np.diff(df_col1))) < 0).nonzero()[0] + 1         # local max
    dict_ = {'loc_min': loc_min, 'loc_max': loc_max}

    return dict_

def computeInterval_peak_ave(pH_loc_min_series):
    '''
    Code that computes the duration between local minimum of the pH trend.
    Returns the average of the duration in minutes.
    
    Parameter: pH_loc_min_series is the pH reading corresponding to the local minima whose index is the DATETIME
    '''
    ser_TK03_pH_A = pH_loc_min_series
    time_diff_between_peaks = np.diff(ser_TK03_pH_A.index) # get the difference between the peaks in the pH reading series from Pi
    time_diff_bw_peaks_minutes = time_diff_between_peaks/np.timedelta64(1, 'm') # convert the units from nanoseconds to minutes
    
    return time_diff_bw_peaks_minutes.mean()

In [4]:
raw_108_Pi = pd.read_excel(r'C:\Users\v.t.flores\Documents\FNTRL_Pi_pH.xlsx') #Load the Pidata from the excel file
raw_108_Pi = raw_108_Pi.set_index('DATETIME')
raw_108_Pi = convert_entire_DF_to_numeric(raw_108_Pi)[1:] # select the all the rows except the first

tk03_pH_A = raw_108_Pi['108TK03_pH_A'] # select the 108TK03 pH
Dict_ = getTheLocalMinMax(tk03_pH_A) # use the function to get the index where there is local_MinMax

ser_TK03_pH_A = raw_108_Pi.iloc[Dict_['loc_min']]['108TK03_pH_A'] # using the index from Dict_ get the values for pH for any tank you select (e.g. 108TK03_pH_A)
mean_interval_length = computeInterval_peak_ave(ser_TK03_pH_A) # Using the function computeInterval_peak_ave, compute the average interval of the local minima

In [1]:

#REFERENCE FUNCTION

def getTheLocalMinMax(df_col1, df_col2):
    df_col1 = pd.to_numeric(df_col1, errors='coerce')
    b = (np.diff(np.sign(np.diff(df_col1))) > 0).nonzero()[0] + 1         # local min
    df_col2 = pd.to_numeric(df_col2, errors='coerce')
    f = (np.diff(np.sign(np.diff(df_col2))) < 0).nonzero()[0] + 1         # local max

    try:
        ft_cyc_lim = {}
        for i in range(0, len(b)):
            for j in range(0, len(f)):
                if f[j] > b[i] and f[j] < b[i+1]:
                    if f[j+1]<b[i+1]:
                        cyc_start = b[i]
                        cyc_end = f[j+1]
                        ft_cyc_lim.setdefault(cyc_start,cyc_end)
    except IndexError:
        print('\nDone')
    return ft_cyc_lim
